In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter   

from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pack_sequence,pad_packed_sequence
from sklearn.preprocessing import LabelEncoder
import joblib
import librosa
import os

In [2]:
class MyDataset(Dataset):
    def __init__(self, trainX, trainY):
        self.trainX = trainX
        self.trainY = trainY
    
    def __getitem__(self, idx):
        # return torch.FloatTensor(self.trainX[idx]), torch.FloatTensor(self.trainY[idx])
        return self.trainX[idx], self.trainY[idx]
        
    def __len__(self):
        assert np.array(self.trainX).shape[0] == np.array(self.trainY).shape[0]
        return np.array(self.trainX).shape[0]

In [3]:
def compute_accuracy(output, target):
    target_hat = torch.max(output, dim=1).indices
    correct_num = int(sum(target_hat==target))
    total_num = target.shape[0]
    return correct_num, total_num

In [4]:
def test(model, data_loader):
    # accuracy
    with torch.no_grad():
        correct_list = []
        total_list = []
        for x, y in tqdm(data_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = lstm(x, device)
            correct_num, total_num = compute_accuracy(outputs, y)
            correct_list.append(correct_num)
            total_list.append(total_num)
        
        accuracy = sum(correct_list) / sum(total_list)
        
 
    return accuracy

In [5]:
def compute_F1(model, data_loader):
    
    labelencoder = joblib.load('labelencoder.pkl')

    with torch.no_grad():
       
        matrix_dict = {}
        
        for x, y in tqdm(data_loader):
            x = x.to(device)
            y = y.to(device)

            outputs = lstm(x, device)
    
            target_hat = str(int(torch.max(outputs, dim=1).indices))
            target = str(int(y))
            if target_hat not in matrix_dict:
                matrix_dict[target_hat] = {'TP':0, 'FP':0, 'FN':0}
            if target not in matrix_dict:
                matrix_dict[target] = {'TP':0, 'FP':0, 'FN':0}
                
            if target_hat == target:
                matrix_dict[target_hat]['TP'] += 1
            else:
                matrix_dict[target_hat]['FP'] += 1
                matrix_dict[target]['FN'] += 1
                
        f1_dict = {}
        for encoded_label in matrix_dict:
            true_label = labelencoder.inverse_transform([int(encoded_label)])[0]
            TP = matrix_dict[encoded_label]['TP']
            FP = matrix_dict[encoded_label]['FP']
            FN = matrix_dict[encoded_label]['FN']
            
            if TP == 0:
                precision =0
                recall = 0
                F1_score = 0
            else:
                precision = TP/(TP+FP)
                recall = TP/(TP+FN)
                F1_score = 2 * (precision * recall)/(precision + recall)            
    
            f1_dict[true_label] = {'precision':precision, 'recall':recall, 'f1-score':F1_score}
            
    # Macro-F1 : an arithmetic mean of the per-class F1-scores
    sum_F1 = 0
    for label in f1_dict:
        sum_F1 += f1_dict[label]['f1-score']
    macro_f1 = sum_F1 / len(f1_dict)
    
    return macro_f1, f1_dict

In [13]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, batch_size, bi):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.bi = 2 if bi == True else 1
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=bi)
        
        self.fc = nn.Linear(self.bi*hidden_size, num_classes)
        # self.softmax = nn.Softmax(dim=1)



    def forward(self, x, device):
        h_0 = Variable(torch.zeros(
            self.num_layers * self.bi, self.batch_size, self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers* self.bi, self.batch_size, self.hidden_size))
        
        h_0 = h_0.to(device)
        c_0 = c_0.to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        # h_out = h_out.view(-1, self.hidden_size)
        h_out = ula[:, -1, :]
        # h_out = h_out.view(-1, self.hidden_size)


        
        out = self.fc(h_out)
        # out = self.softmax(out)
        
        return out

In [14]:
base_path = 'augmented_mfcc24_npy_files/'
with open(base_path+'testX.npy', 'rb') as f:
    testX = np.load(f, allow_pickle=True)
with open(base_path+'testY.npy', 'rb') as f:
    testY = np.load(f, allow_pickle=True)
    
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
train_post_competition = pd.read_csv('augmented_train_post_competition.csv')
data_loader_test = DataLoader(MyDataset(testX, testY), batch_size=1, shuffle=True, drop_last=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
lstm = torch.load('ckp/cnn_3conv_3linear_20epoch_0.01lr.ckp')

print('Testing')
test_accuracy = test(lstm, data_loader_test)
macro_f1, f1_dict = compute_F1(lstm, data_loader_test)
print('Test accuracy: {0:.2f}'.format(test_accuracy))
print('Test F1-score : {0:.2f}'.format(macro_f1))

AttributeError: Can't get attribute 'CNN' on <module '__main__'>

In [ ]:
# precision-recall-f1 matric

pd.merge(pd.DataFrame.from_dict(f1_dict).T, pd.DataFrame(train_post_competition.label.value_counts()), left_index=True, right_index=True)

In [ ]:
class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.cnn_layers = Sequential(
            # define a 2D convolutional layer
            Conv2d(1, 32, kernel_size=(5, 5), stride=1, padding=(0,0)),
            ReLU(inplace=True),
            BatchNorm2d(32),
            MaxPool2d(kernel_size=2, stride=2),

            # Defining another 2D convolution layer
            Conv2d(32, 16, kernel_size=(3, 3), stride=1),
            ReLU(inplace=True),
            # BatchNorm2d(16),
            # MaxPool2d(kernel_size=2, stride=2),

            # Defining another 2D convolution layer
            Conv2d(16, 8, kernel_size=(3, 3), stride=1),
            ReLU(inplace=True),
            BatchNorm2d(8),
        )
                

        self.linear_layers = Sequential(
            
            
            # Linear(3839840, 10000),
            # Linear(10000, 1000),
            
            Linear(1000,256),
            BatchNorm1d(256),
            ReLU(inplace=True),
            # Dropout(0.2),
            
            Linear(256, 128),
            BatchNorm1d(128),
            ReLU(inplace=True),
            # Dropout(0.1),
            
            Linear(128, 41),
            Softmax(dim=1)
        )
        
        
    # Defining the forward pass
    def forward(self, x):
        # x = self.cnn_layers(x)
        x = self.vgg16(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        # print(x.shape)
        # print(torch.max(x, dim=1))
        return x